In [2]:
import pandas as pd
import numpy as np
import keras
# import matplotlib.pyplot as plt
import time

In [76]:
new_data_x = np.load("new_data_x.npy")
new_data_x.shape

(4000, 10, 81)

In [77]:
from keras.utils import to_categorical

new_data_x = to_categorical(new_data_x, num_classes=10)
new_data_x.shape

(4000, 10, 81, 10)

In [13]:
dt = pd.read_csv("sudoku_dataset/sudoku.csv")
print(dt.shape)

(1000000, 2)


In [9]:
from keras.utils import to_categorical

def preprocessing_data(data):
    array = [list(map(int,list(i))) for i in data]
    return to_categorical(array, num_classes=10)

In [78]:
train_size = 3900
test_size = 100
epochs = 10
batch = 32

In [79]:
new_data_x = np.reshape(new_data_x, (new_data_x.shape[0], new_data_x.shape[1], -1))
new_data_x.shape

(4000, 10, 810)

In [80]:
x_train = new_data_x[:train_size]
y_train = preprocessing_data(dt["solutions"][:train_size])

x_test = new_data_x[train_size:train_size+test_size]
y_test = preprocessing_data(dt["solutions"][train_size:train_size+test_size])

print(x_train.shape)
# print(y_train)

(3900, 10, 810)


In [81]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Reshape, Dropout, BatchNormalization, Activation

model = Sequential()
model.add(LSTM(810, input_shape=x_train.shape[1:]))
model.add(Dense(810))
model.add(Reshape((81,10)))
model.add(Activation('softmax'))

model.summary()

model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['accuracy'])

tbCallBack = keras.callbacks.TensorBoard(log_dir='./logs/{}'.format(int(time.time())), histogram_freq=0, write_graph=True, write_images=True)

history = model.fit(x_train, y_train, validation_split=0.125, epochs=epochs, batch_size=batch, callbacks=[tbCallBack])

score = model.evaluate(x_test, y_test, batch_size=batch)
print(score)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_13 (LSTM)               (None, 810)               5252040   
_________________________________________________________________
dense_12 (Dense)             (None, 810)               656910    
_________________________________________________________________
reshape_12 (Reshape)         (None, 81, 10)            0         
_________________________________________________________________
activation_10 (Activation)   (None, 81, 10)            0         
Total params: 5,908,950
Trainable params: 5,908,950
Non-trainable params: 0
_________________________________________________________________
Train on 3412 samples, validate on 488 samples
Epoch 1/10
3412/3412 [==============================] - 368s 108ms/step - loss: 0.0622 - acc: 0.5526 - val_loss: 0.0338 - val_acc: 0.8533
Epoch 2/10
3412/3412 [==============================] - 332s 97ms/step - loss: 

In [82]:
x_train_1 = np.reshape(x_test[0], (1,10,810))
y_test_1 = model.predict(x_train_1)
y_test_1 = np.argmax(y_test_1[0], axis=1)

In [83]:
for i,j in zip(y_test_1,np.argmax(y_test[0], axis=1)):
    if i!=j:
        print(i,j)

6 5
